In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
# imports

import pandas as pd
from google.colab import files
from os import listdir
from os.path import isfile, join
import re
from collections import Counter
from PIL import Image
import numpy as np
from keras.models import Model
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout
from matplotlib.image import imread
from random import shuffle
from math import ceil

In [3]:
# variaveis

img_to_load = 300 # imagens a serem carregadas por vez (carregando as 10000 de uma vez sobrecarrega o sistema)
crop_size = 150 # altura e largura da imagem a ser enviada para o modelo

In [4]:
#função para pegar só o centro da imagem

def crop_center(img,cropx,cropy):
    y,x = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return img[starty:starty+cropy,startx:startx+cropx]

In [ ]:
# usar se estiver usando google colab, isso descompacta um arquivo no diretório do notebook

!unzip regression_model_0_0070226178504526615.zip

Archive:  regression_model_0_0070226178504526615.zip
   creating: regression_model_0_0070226178504526615/
   creating: regression_model_0_0070226178504526615/regression_model/
   creating: regression_model_0_0070226178504526615/regression_model/assets/
  inflating: regression_model_0_0070226178504526615/regression_model/saved_model.pb  
   creating: regression_model_0_0070226178504526615/regression_model/variables/
  inflating: regression_model_0_0070226178504526615/regression_model/variables/variables.data-00000-of-00001  
  inflating: regression_model_0_0070226178504526615/regression_model/variables/variables.index  


In [5]:
# carrega um modelo salvo anteriormente

c_model = load_model('classification_model/classification_model')
r_model = load_model('regression_model_0_0070226178504526615/regression_model')

In [6]:
public_path = '/gdrive/My Drive/IDAO/public_test/'
public_files = [f'{public_path}{f}' for f in listdir(public_path) if isfile(join(public_path, f))]
print(len(public_files))
public_files[:5]

1502


['/gdrive/My Drive/IDAO/public_test/c8bba7ea2625a956b2a47525026768a1ea141eec.png',
 '/gdrive/My Drive/IDAO/public_test/dd5cc2be5d11345a1741141808820d724f086ede.png',
 '/gdrive/My Drive/IDAO/public_test/6d43cd6fd077e9963b0a309edf9d171da775452c.png',
 '/gdrive/My Drive/IDAO/public_test/1469ef19451b127e244714aa20c404d0d787060d.png',
 '/gdrive/My Drive/IDAO/public_test/0645a92da4ffb6a99968f1033da4274387de1786.png']

In [7]:
private_path = '/gdrive/My Drive/IDAO/private_test/'
private_files = [f'{private_path}{f}' for f in listdir(private_path) if isfile(join(private_path, f))]
print(len(private_files))
private_files[:5]

15062


['/gdrive/My Drive/IDAO/private_test/0b791e70cba92d12e04361ab6719c7f261e8591d.png',
 '/gdrive/My Drive/IDAO/private_test/a0078020a12a4b565e9afc19c115100a8689652f.png',
 '/gdrive/My Drive/IDAO/private_test/a13041926557e99fdbd78eeb53a5eb96711124f6.png',
 '/gdrive/My Drive/IDAO/private_test/352849a955094f76fe71b0a639707ebc374fa937.png',
 '/gdrive/My Drive/IDAO/private_test/77c0573439b286e23ffea0a6a93fbe0c4a03a76b.png']

In [8]:
all_test_files = public_files + private_files
len(all_test_files)

16564

In [ ]:
#img_to_load = 10

df = pd.DataFrame()

for i in range(ceil(len(all_test_files)/img_to_load)):
    begin = i*img_to_load
    limit = min(i*img_to_load + img_to_load, len(all_test_files))
    if begin >= limit:
        break

    X = [crop_center(imread(file), crop_size, crop_size) for file in all_test_files[begin:limit]]
    X = np.stack(X, axis=0)
    X = X.reshape((X.shape + (1,)))
    print(f'loaded {begin} to {limit}')
    
    c_predictions = (c_model.predict(X) > 0.5).astype("int32")
    r_predictions = (r_model.predict(X)) * 30

    list_dict = [{'id': re.findall(r"(?<=_test\/).*(?=\.png)", all_test_files[i])[0], 
                  'classification_predictions': c_predictions[i][0], 
                  'regression_predictions': r_predictions[i][0]} for i in range(10)]
    df_aux = pd.DataFrame(list_dict)
    df = pd.concat([df, df_aux])

loaded 0 to 300
loaded 300 to 600
loaded 600 to 900
loaded 900 to 1200
loaded 1200 to 1500
loaded 1500 to 1800
loaded 1800 to 2100
loaded 2100 to 2400
loaded 2400 to 2700
loaded 2700 to 3000
loaded 3000 to 3300
loaded 3300 to 3600
loaded 3600 to 3900
loaded 3900 to 4200
loaded 4200 to 4500
loaded 4500 to 4800
loaded 4800 to 5100
loaded 5100 to 5400


In [ ]:
df.to_csv('predictions.csv', index=False)

In [ ]:
df

,id,classification_predictions,regression_predictions
0,c8bba7ea2625a956b2a47525026768a1ea141eec,0,29.879734
1,dd5cc2be5d11345a1741141808820d724f086ede,0,29.717144
2,6d43cd6fd077e9963b0a309edf9d171da775452c,0,9.943920
3,1469ef19451b127e244714aa20c404d0d787060d,0,10.039680
4,0645a92da4ffb6a99968f1033da4274387de1786,0,9.951762
5,1c05feeece98be0c3fb6211566425a0d0676305a,0,10.151268
6,b29c8d3b8027a6216f124e2fc2ff12b5040a09c7,0,3.034326
7,f59f9027ef68dd8472cbbe2cc89d220fa2245bab,0,29.791040
8,3eb403775f597f48670f0eb8a38cde306c48e875,0,10.075497
9,c3b6bb18dfa6bba38c8e306c594f79eb563d6ffd,0,29.826731
